In [ ]:
from app.utilities import *
import ibis
from ibis import _
con = ibis.duckdb.connect("duck.db", extensions=['httpfs', 'spatial', 'h3'])
set_secrets(con) # s3 credentials
set_aws_secrets(con)

#con.raw_sql("SET memory_limit = '20GB';")
#con.raw_sql("set threads=40;")


In [ ]:
gbif = con.read_parquet("s3://cboettig/gbif/2024-10-01/**")




In [ ]:
(con
 .read_geo("/vsicurl/https://data.source.coop/cboettig/us-boundaries/mappinginequality.json")
 .to_parquet("s3://cboettig/gbif/mappinginequality.parquet")
)

In [ ]:
import minio
import streamlit as st
from pathlib import Path
minio_key = st.secrets["MINIO_KEY"]
minio_secret = st.secrets["MINIO_SECRET"]
mc = minio.Minio("minio.carlboettiger.info", minio_key, minio_secret)

obj = mc.list_objects("cboettig", "gbif/redlined/", recursive=True)

finished_cities = [str(Path(i.object_name).stem) for i in obj if not i.is_dir]
finished_cities



In [ ]:

mappinginequality = con.read_parquet("s3://cboettig/gbif/mappinginequality.parquet")
cities = mappinginequality.select(_.city).distinct().order_by(_.city).execute()["city"]


remaining_cities = set(cities) - set(finished_cities)
remaining_cities

In [ ]:
%%time 

for city in remaining_cities:
    print(city)
    gdf = (mappinginequality
           .filter(_.city == city)
           .mutate(area = _.geom.area())
#           .agg(geom = _.geom.unary_union())
    )
    
    bounds =  gdf.execute().total_bounds
    points = (gbif
           .filter(_.decimallongitude >= bounds[0], 
                   _.decimallongitude < bounds[2], 
                   _.decimallatitude >= bounds[1], 
                   _.decimallatitude < bounds[3])
             )
                  
    (gdf
     .join(points, gdf.geom.intersects(points.geom))
     .to_parquet(f"s3://cboettig/gbif/redlined/{city}.parquet")
    )


In [ ]:
%%time

points = gbif
city = (con
        .read_geo("/vsicurl/https://data.source.coop/cboettig/us-boundaries/mappinginequality.json")
        .mutate(area = _.geom.area())
       )

(city
 .join(points, city.geom.intersects(points.geom))
 .to_parquet("s3://cboettig/gbif/redlined-city-gbif.parquet")
)

In [ ]:

overture = con.read_parquet('s3://overturemaps-us-west-2/release/2024-11-13.0/theme=divisions/type=division_area/*', filename=True, hive_partitioning=1)
usa = overture.filter(_.subtype=="country").filter(_.country == "US").select(_.geometry).execute()
ca = (overture
       .filter(_.country == "US", _.subtype == "region")
       .select('region', 'geometry')
       .filter(_.region == "US-CA")
       .execute()
      )

In [ ]:
%%time
#points = con.read_parquet("s3://cboettig/gbif/cache/tmp_ca.parquet", "points").filter(_["class"] == "Aves")
points = gbif

city = (con
        .read_geo("/vsicurl/https://data.source.coop/cboettig/us-boundaries/mappinginequality.json")
        .filter(_.city == "Los Angeles")
        .mutate(area = _.geom.area())
       )

(city
 .join(points, city.geom.intersects(points.geom)) 
 .group_by(_.grade)
 .agg(n = _.count(),
      area = _.area.sum())
 .mutate(density = _.n /_.area)
 .order_by(_.density.desc())
 .execute()
)



In [ ]:
(city
 .join(points, city.geom.intersects(points.geom)) 
 .group_by(_.area_id, _.grade)
 .agg(n = _.count(),
      area = _.area.sum())
 .mutate(density = _.n /_.area)
 .group_by(_.grade)
 .agg(mean = _.density.mean(),
        sd = _.density.std())
 .order_by(_.mean.desc())
 .execute()
)

In [ ]:
import streamlit as st
minio_key = st.secrets["MINIO_KEY"]
minio_secret = st.secrets["MINIO_SECRET"]
query=   f'''
CREATE OR REPLACE SECRET secret2 (
    TYPE S3,
    KEY_ID '{minio_key}',
    SECRET '{minio_secret}',
    ENDPOINT 'minio.carlboettiger.info',
    URL_STYLE 'path',
    SCOPE 's3://cboettig/gbif'
);
'''

In [ ]:
city

In [ ]:
import duckdb
db = duckdb.connect()
db.sql(query)
df = db.sql(f'''
LOAD SPATIAL;

CREATE OR REPLACE VIEW points as SELECT * FROM 's3://cboettig/gbif/cache/tmp.parquet';

CREATE OR REPLACE VIEW area as SELECT *
FROM (
  SELECT
    *
  FROM st_read('/vsicurl/https://data.source.coop/cboettig/us-boundaries/mappinginequality.json') AS "t0"
  WHERE
    "t0"."city" = 'Oakland'
);
''')


# CREATE INDEX my_idx ON my_table USING RTREE (geom);




db.sql(f'''            
SELECT *
FROM area, points 
WHERE st_intersects(area.geom, points.geom)
LIMIT 100
''')

# each point is now joined to it's containing area!
df.to_df()

In [ ]:


#CREATE INDEX idx_points_geom_4326 ON points USING gist (ST_Transform(point,4326));


In [ ]:
# Overture-based polygons

overture = con.read_parquet('s3://overturemaps-us-west-2/release/2024-11-13.0/theme=divisions/type=division_area/*', filename=True, hive_partitioning=1)
#gdf = overture.filter(_.subtype=="country").filter(_.country == "US").select(geometry)
gdf = (overture
       .filter(_.country == "US", _.subtype == "region")
       .select('region', 'geometry')
       .filter(_.region == "US-MA")
       .execute()
      )
gdf

In [ ]:
%%time

## export in gdal formats?  not working?

bucket = "cboettig/gbif"
dest2 = "cache/tmp.geojson"
query = ibis.to_sql(sel)

#con.raw_sql(f"COPY ({query}) TO 's3://{bucket}/{dest2}'  WITH (FORMAT GDAL, DRIVER 'FlatGeoBuf');")
#con.raw_sql(f"COPY ({query}) TO 's3://{bucket}/{dest2}'  WITH  (FORMAT GDAL, DRIVER 'GeoJSON', LAYER_CREATION_OPTIONS 'WRITE_BBOX=YES');")
#con.raw_sql(f"COPY ({query}) TO 'tmp.geojson'  WITH  (FORMAT GDAL, DRIVER 'GeoJSON', LAYER_CREATION_OPTIONS 'WRITE_BBOX=YES');")
#con.sql('SELECT * FROM ST_DRIVERS();').execute()
